In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: $SPARK_VERSION-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
You should consider upgrading via the 'C:\Users\saadi\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'findspark'

In [4]:
pip  install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Demographics").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/nflx-data-science-adv/week-4/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("demographics.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|    68| South Dakota|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|    73|     Delaware|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|    69| South Dakota|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|    83|West Virginia|
|  5|        Myung Brewer| 20|        1.68|     

In [4]:
# Print the column names
df.columns

['id',
 'name',
 'age',
 'height_meter',
 'weight_kg',
 'children',
 'occupation',
 'academic_degree',
 'salary',
 'location']

In [5]:
# Print out the first 10 rows
df.show(10)

+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|    68| South Dakota|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|    73|     Delaware|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|    69| South Dakota|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|    83|West Virginia|
|  5|        Myung Brewer| 20|        1.68|     

In [6]:
# Select the age, height_meter, and weight_kg columns and use describe to show the summary statistics
df.select(["age", "height_meter", "weight_kg"]).describe().show()

+-------+------------------+------------------+------------------+
|summary|               age|      height_meter|         weight_kg|
+-------+------------------+------------------+------------------+
|  count|              1000|              1000|              1000|
|   mean|            42.933|1.7519499999999995|            64.011|
| stddev|14.255445581556843|0.1436897499623555|15.005733939099779|
|    min|                18|               1.5|                38|
|    max|                67|                 2|                90|
+-------+------------------+------------------+------------------+



In [7]:
# Print the schema to see the types
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- height_meter: string (nullable = true)
 |-- weight_kg: string (nullable = true)
 |-- children: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- academic_degree: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- location: string (nullable = true)



In [8]:
# Rename the Salary column to `Salary (1k)` and show only this new column
df = df.withColumnRenamed('Salary', 'Salary (1k)')
df.select("Salary (1k)").show()

+-----------+
|Salary (1k)|
+-----------+
|         68|
|         73|
|         69|
|         88|
|         83|
|         65|
|         72|
|         65|
|         87|
|         72|
|         73|
|         90|
|         78|
|         69|
|         75|
|         77|
|         76|
|         90|
|         79|
|         77|
+-----------+
only showing top 20 rows



In [9]:
# Create a new column called `Salary` where the values are the `Salary (1k)` * 1000
# Show the columns `Salary` and `Salary (1k)`
df = df.withColumn("Salary", df["Salary (1k)"] * 1000)
df.select(["Salary", "Salary (1k)"]).show()

+-------+-----------+
| Salary|Salary (1k)|
+-------+-----------+
|68000.0|         68|
|73000.0|         73|
|69000.0|         69|
|88000.0|         88|
|83000.0|         83|
|65000.0|         65|
|72000.0|         72|
|65000.0|         65|
|87000.0|         87|
|72000.0|         72|
|73000.0|         73|
|90000.0|         90|
|78000.0|         78|
|69000.0|         69|
|75000.0|         75|
|77000.0|         77|
|76000.0|         76|
|90000.0|         90|
|79000.0|         79|
|77000.0|         77|
+-------+-----------+
only showing top 20 rows

